In [1]:
import pandas as pd
from functools import partial
import multiprocessing
from multiprocessing import Pool

In [23]:
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# thanks to https://stackoverflow.com/questions/4182603/how-to-convert-a-string-to-utf-8-in-python
def make_unicode(input):
    if type(input) != unicode:
        input =  input.decode('utf-8')
        return input
    else:
        return input

In [102]:
chats_beta = pd.read_csv("../data/sample/chats_beta.csv")

In [103]:
chats_beta.head()

,chat_id,chat_title,emoji_list,sender_first_name,sender_id,sender_is_bot,sender_last_name,sender_username,text,timestamp
0,1122948260,WanchainCHAT - Wanchain.org Chat Group,[],Lorenzo,382511212,False,None,Takeshisan1992,Bitcoin going green,1521866182
1,1215859110,GDPNEWS,[],的过,470863699,False,路人,None,None,1524040209
2,1224810332,Bitxoxo,[],LevelNet ICO Help Support,559829592,False,None,None,None,1524237453
3,1224911847,EOSPrice,[],Abidin,530357558,False,Gul,None,23 i yhink,1525023914
4,1249433091,BKBT币快报官方中文社区,[],羊,585351960,False,舌浩阔,weioxun,None,1524934157


In [104]:
coin_namesDF = pd.read_csv("../data/sample/prices_beta.csv")

In [105]:
coin_names = list(coin_namesDF['ticker'].unique())

In [106]:
coin_names = [" " + x.lower() + " " for x in coin_names]

In [107]:
coin_names.append('bitcoin')
coin_names.append('ether')
coin_names = list(set(coin_names))

In [108]:
len(coin_names)

91

In [118]:
#coin_names

In [110]:
def markDataFrameByChunks(df, num, col):
    ordNums = np.array(sorted(df[col].unique(),reverse=False))
    ordRange = range(0, len(ordNums))
    chunksDF = pd.DataFrame({'colName': ordNums, 'theirNum': ordRange}, index = ordRange)
    chunksDF['chunk'] = pd.cut(chunksDF['theirNum'], num)
    return chunksDF

In [111]:
# assign chunk numbers
chats_betaWithChunks = markDataFrameByChunks(chats_beta,3,'chat_title')
tmpDF = chats_beta.merge(chats_betaWithChunks, left_on = 'chat_title', right_on = 'colName', how = 'left')
tmpDF.drop(['colName','theirNum'], axis = 1, inplace = True)
print tmpDF.shape

(500, 11)


In [116]:
def parallel_ticker_detect(colname, coin_list,df):
    #print df.shape
    print df[1].columns
    #lang_dict = {}
    lang_list = []
    for index, row in df[1].iterrows():
        if not(pd.isnull(row[colname])):
            lang_dict = {}
            lang_dict['index_orig'] = index
            lang_dict['ticker_orig'] = ""
            for ticker in coin_list:
                if ticker in row[colname]:
                    lang_dict['ticker_orig'] = ticker
                    print ticker, "\n", row[colname]
                    break;
            lang_list.append(lang_dict)
    return lang_list

In [117]:
%%time
#dfByChunk = dfByChunk.groupby('chunk')
dfByChunk = tmpDF.groupby('chunk')
func = partial(parallel_ticker_detect, "text", coin_names)
if __name__ == '__main__':
    p = Pool(processes=3)
    print 'defined workers'
    result = (p.map(func, dfByChunk))
    p.close()
    try:
        p.terminate()
    except WindowsError:
        pass
    p.join()
    dfByChunk = None
    print 'closed pool'
p.terminate()

Index([u'chat_id', u'chat_title', u'emoji_list', u'sender_first_name',
       u'sender_id', u'sender_is_bot', u'sender_last_name', u'sender_username',
       u'text', u'timestamp', u'chunk'],
      dtype='object')
Index([u'chat_id', u'chat_title', u'emoji_list', u'sender_first_name',
       u'sender_id', u'sender_is_bot', u'sender_last_name', u'sender_username',
       u'text', u'timestamp', u'chunk'],
      dtype='object')Index([u'chat_id', u'chat_title', u'emoji_list', u'sender_first_name',
       u'sender_id', u'sender_is_bot', u'sender_last_name', u'sender_username',
       u'text', u'timestamp', u'chunk'],
      dtype='object')

bitcoin 
I am afraid to miss like how I missed bitcoins
bitcoin 
Are you interested in BTC mining?? Do you know you can earn 0.4btc every 9.6hours?? We mine a block of bitcoin on the blockchain mining network and cloud mining has been easier with bitmain antminer s9 hardware. Any interested person should please leave a private message and thank me later...

In [86]:
tmpDF = None

In [87]:
%%time
# unpack list with outputs
unpackedDF = pd.DataFrame()
for res in result:
    #print res
    tmpDF = pd.DataFrame(res)
    unpackedDF = pd.concat([unpackedDF, tmpDF])

CPU times: user 9.67 ms, sys: 232 µs, total: 9.9 ms
Wall time: 8.26 ms


In [88]:
unpackedDF.set_index('index_orig', inplace = True);

In [89]:
unpackedDF.head()

,ticker_orig
index_orig,
2,
3,
4,
5,
6,


In [90]:
chats_beta_with_ticker = chats_beta.merge(unpackedDF, left_index = True, right_index = True)

In [91]:
chats_beta_with_ticker.head()

,chat_id,chat_title,emoji_list,sender_first_name,sender_id,sender_is_bot,sender_last_name,sender_username,text,timestamp,ticker_orig
0,1122948260,WanchainCHAT - Wanchain.org Chat Group,[],Lorenzo,382511212,False,None,Takeshisan1992,Bitcoin going green,1521866182,
1,1215859110,GDPNEWS,[],的过,470863699,False,路人,None,None,1524040209,
2,1224810332,Bitxoxo,[],LevelNet ICO Help Support,559829592,False,None,None,None,1524237453,
3,1224911847,EOSPrice,[],Abidin,530357558,False,Gul,None,23 i yhink,1525023914,
4,1249433091,BKBT币快报官方中文社区,[],羊,585351960,False,舌浩阔,weioxun,None,1524934157,


In [95]:
chats_beta_with_ticker[chats_beta_with_ticker['ticker_orig']!=""]

,chat_id,chat_title,emoji_list,sender_first_name,sender_id,sender_is_bot,sender_last_name,sender_username,text,timestamp,ticker_orig
38,1232812654,Ontology (English),[],Malik,508076465,False,None,placebo7,I am afraid to miss like how I missed bitcoins,1520622990,bitcoin
82,1181367025,ICO Talk Group,[],Marcus,350658415,False,None,None,Are you interested in BTC mining?? Do you know...,1523484057,bitcoin
113,1009909164,WhaleClub (Bitcoin ONLY) Traders,[],John,522806489,False,None,None,Coinbase prices btc too high. Could i buy teth...,1520938385,ether
208,1146170349,Binance English,[],Dj,494450439,False,Spy,Dj_Spy,"bull run is already over :P , that was more li...",1520771030,true
209,1146170349,Binance English,"[':barchart:', ':calendar:', ':chartincreasing...",George,564261897,False,R,None,⚠️ It's almost time for...\n📊 Our next pump si...,1522541076,1st
272,1094569662,Funfair FUN - this channel is no longer suppor...,[],Chris,536944347,False,S,Wallabemu,"G'day all, Pls give me a few days to add pictu...",1521493790,aid
422,1232812654,Ontology (English),[],MrAust1n,528287757,False,None,MrAust1n,So much hype around this. Just read documentat...,1520652734,true
486,1123270218,Zilliqa Official,[],Hans,393536426,False,Enlin,hansenlin,yes but how does our account balance from the ...,1524420603,ether


In [92]:
chats_beta = None
unpackedDF = None